In [1]:
def draw_Kline(stock_code,stock_name,save_dir):
    test_file = "%s/%s.csv" %(history_data_dir,stock_code)
    df = pd.read_csv(test_file)
    df.rename(columns={"vol":"volume"},inplace=True)
    # date必须作为index，且格式需要调整
    df["trade_date"] = pd.to_datetime(df["trade_date"],format='%Y%m%d')
    df.set_index("trade_date", inplace=True)

    # 符合中国习惯的配色风格——红涨绿跌自然是必须实现的第一步
    my_color = mpl.make_marketcolors(up='r',
                                     down='g',
                                     edge='inherit',
                                     wick='inherit',
                                     volume='inherit')
    # 设置图表的背景色
    my_style = mpl.make_mpf_style(marketcolors=my_color,
                                  figcolor='(0.82, 0.83, 0.85)',
                                  gridcolor='(0.82, 0.83, 0.85)')

    df["macd_fast"],df["macd_slow"],df["macd_volume"] = macd(df)
    plot_df = df.iloc[-200:,]
    # 增加图表
    add_plot = [mpl.make_addplot(plot_df["macd_fast"], color="red",panel=2),
                mpl.make_addplot(plot_df["macd_slow"], color="green",panel=2),
                mpl.make_addplot(plot_df["macd_volume"], type='bar', width=0.7,panel=2)]
    save = '%s/%s.jpg' %(save_dir,stock_name)
    # type指定图类型，addplot增加图，mav增加均线,savefig做存储
    mpl.plot(plot_df, type = 'candle',style=my_style,figsize=(12, 8),title=stock_name,addplot=add_plot,
                 mav = (5, 10, 20, 30,60,120),volume = True, savefig=save)